# LangGraph Graph Construction: Nodes & Edges

Build your first LangGraph workflow by connecting nodes and edges into a graph.

## Learning Objectives

By the end of this notebook, you will:

1. **Understand `MessagesState`** — How LangGraph automates the conversation history you managed manually in the previous notebook
2. **Define nodes** — Create the LLM node and tool node that perform the work
3. **Connect edges** — Wire nodes together with edges and conditional edges to control the flow
4. **Visualize the graph** — See the complete workflow as a diagram

## 1. Environment Setup

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../../.env")
print("✅ Environment loaded")

In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from typing import Literal

print("✅ All imports successful")

In [ ]:
# Mermaid helper for graph visualization
def render_mermaid(diagram_code, width=400):
    """Render Mermaid diagrams using mermaid.ink service."""
    from IPython.display import Image, display
    import base64
    
    graphbytes = diagram_code.encode('utf-8')
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode('ascii')
    url = f'https://mermaid.ink/img/{base64_string}'
    display(Image(url=url, width=width))

print("✅ Visualization helper defined")

## 2. From Manual History to MessagesState

In the previous notebook, we managed conversation history manually using a Python list:

```python
conversation = []
conversation.append(HumanMessage(content="..."))
response = llm.invoke(conversation)
conversation.append(response)  # Must remember to do this!
```

LangGraph's `MessagesState` automates this. It is essentially a typed dictionary with a single key `"messages"` that **automatically accumulates** messages as they flow through the graph.

```python
# MessagesState is equivalent to:
{"messages": [HumanMessage, AIMessage, ToolMessage, ...]}
```

When a node returns `{"messages": [new_message]}`, LangGraph **appends** it to the existing list rather than replacing it. This is the key difference — you no longer need to manage the list yourself.

## 3. Define Tools

We'll reuse the tools from our earlier notebooks.

In [ ]:
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates:
        return f"Error: Unsupported currency {from_currency}"
    if to_currency not in exchange_rates:
        return f"Error: Unsupported currency {to_currency}"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    """
    if principal <= 0:
        return "Error: Principal must be greater than 0"
    if annual_interest_rate < 0:
        return "Error: Interest rate cannot be negative"
    if tenure_months <= 0:
        return "Error: Tenure must be greater than 0"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

## 4. Initialize LLM with Tools

`bind_tools()` tells the LLM about the available tools so it can decide when to call them.

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

tools = [currency_converter, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM initialized with tools")

## 5. Define Nodes

A **node** is a function that takes the current state and returns an update to it.

Our graph needs two nodes:

| Node | Purpose | Implementation |
|---|---|---|
| `llm` | Calls the LLM to generate a response or request a tool call | Custom function |
| `tools` | Executes the tool that the LLM requested | LangGraph's built-in `ToolNode` |

In [ ]:
# Node 1: LLM node
def call_llm(state: MessagesState):
    """LLM node that invokes the LLM with the current messages."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

print("✅ LLM node defined")

Notice the pattern:
- **Input:** `state["messages"]` — reads the full message history from the state
- **Output:** `{"messages": [response]}` — returns the new message to be **appended** to the state

This is exactly what we did manually in the previous notebook, but now LangGraph handles the appending.

In [ ]:
# Node 2: Tool node (built-in)
tool_node = ToolNode(tools)

print("✅ Tool node defined")

`ToolNode` is a prebuilt LangGraph node that:
1. Reads the last AIMessage from state
2. Extracts the `tool_calls` from it
3. Executes the requested tool(s)
4. Returns the result as a `ToolMessage`

## 6. Define the Router

The **router** is a function that decides which node to go to next. It looks at the LLM's response and checks:
- If the LLM wants to call a tool → route to the `tools` node
- If the LLM has a final answer → route to `END`

In [ ]:
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router that decides next step based on whether tool_calls exist."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

print("✅ Router function defined")

## 7. Build the Graph

Now we connect everything together. A LangGraph workflow has three building blocks:

| Component | Method | Purpose |
|---|---|---|
| **State** | `StateGraph(MessagesState)` | Defines the data structure flowing through the graph |
| **Nodes** | `add_node(name, function)` | Processing steps that read/update state |
| **Edges** | `add_edge()` / `add_conditional_edges()` | Connections that control the flow |

### Step 1: Create the graph with state

In [ ]:
workflow = StateGraph(MessagesState)

print("✅ Graph created with MessagesState")

### Step 2: Add nodes

In [ ]:
workflow.add_node("llm", call_llm)
workflow.add_node("tools", tool_node)

print("✅ Nodes added: 'llm' and 'tools'")

### Step 3: Add edges

We need three connections:

1. **START → llm** — Every query begins at the LLM node
2. **llm → tools OR END** — Conditional: depends on whether the LLM wants to call a tool
3. **tools → llm** — After a tool executes, go back to the LLM to process the result

In [ ]:
# Edge 1: Entry point
workflow.add_edge(START, "llm")

print("✅ Edge added: START → llm")

In [ ]:
# Edge 2: Conditional - LLM decides the next step
workflow.add_conditional_edges(
    "llm",
    should_continue,
    {"tools": "tools", END: END}
)

print("✅ Conditional edge added: llm → tools OR END")

In [ ]:
# Edge 3: Loop back after tool execution
workflow.add_edge("tools", "llm")

print("✅ Edge added: tools → llm")

### The complete flow looks like this:

```
START → llm ──────────────────────────────→ END
              │                              ▲
              │ (has tool_calls?)             │ (no tool_calls)
              ▼                              │
            tools ───────────────────────→ llm
```

## 8. Compile and Visualize

Compiling converts the workflow definition into an executable application. It validates that all nodes are reachable and all edges are properly connected.

In [ ]:
app = workflow.compile()

print("✅ Graph compiled successfully")

In [ ]:
# Visualize the graph
mermaid_diagram = app.get_graph().draw_mermaid()
render_mermaid(mermaid_diagram, width=200)

## 9. Quick Test

Let's do a quick test to confirm the graph works. We'll examine the execution in detail in the next notebook.

In [ ]:
# Test with a tool query
result = app.invoke({
    "messages": [HumanMessage(content="Convert 100 USD to EUR")]
})

print(f"Total messages generated: {len(result['messages'])}")
print(f"\nFinal Response:\n{result['messages'][-1].content}")

In [ ]:
# Test with a non-tool query (goes directly to END)
result = app.invoke({
    "messages": [HumanMessage(content="Hello, how are you?")]
})

print(f"Total messages generated: {len(result['messages'])}")
print(f"\nFinal Response:\n{result['messages'][-1].content}")

Notice the difference:
- **Tool query** ("Convert 100 USD to EUR") → Multiple messages (LLM → Tool → LLM)
- **Non-tool query** ("Hello, how are you?") → Only 2 messages (LLM → END directly)

## Conclusion

### What You've Accomplished

✅ **Understood `MessagesState`** — LangGraph's automatic message accumulation replaces manual list management

✅ **Defined nodes** — Created an LLM node (custom function) and a tool node (`ToolNode`)

✅ **Connected edges** — Wired the graph with `add_edge`, `add_conditional_edges`, `START`, and `END`

✅ **Visualized the graph** — Used Mermaid to render the workflow diagram

### Key Concepts

**Nodes** are functions that process state. **Edges** control flow between nodes. **Conditional edges** enable branching logic based on the current state.

### Next Steps

Continue to **Notebook 065c: Graph Compilation & Testing** to understand the complete message flow and test the graph with different scenarios.